### ランダムフォレスト分類器　(RFC)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import load

In [2]:
df = pd.read_csv("../cs3-08/heart_disease-mid.csv")
print(df.shape)
print(df.info())
display(df.head())

(2000, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   HeartDisease     2000 non-null   int64  
 1   BMI              2000 non-null   float64
 2   Smoking          2000 non-null   int64  
 3   AlcoholDrinking  2000 non-null   int64  
 4   Stroke           2000 non-null   int64  
 5   M/F              2000 non-null   object 
 6   AgeCategory      2000 non-null   int64  
 7   GenHealth        2000 non-null   int64  
 8   SleepTime        2000 non-null   float64
dtypes: float64(2), int64(6), object(1)
memory usage: 140.8+ KB
None


,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,M/F,AgeCategory,GenHealth,SleepTime
0,0,24.37,0,0,0,M,0,4,7.0
1,0,41.20,0,0,0,F,1,3,7.0
2,0,19.80,0,0,0,M,10,4,6.0
3,0,24.33,1,0,0,M,8,2,8.0
4,0,19.74,0,0,0,F,0,4,8.0


### 特徴量と正解ラベルを分ける

In [3]:
X = df.drop(columns=  "HeartDisease")
y = df["HeartDisease"]
print("X:", X.shape)
display(X.head())
print("y:", y.shape)
print(y.head())

X: (2000, 8)


,BMI,Smoking,AlcoholDrinking,Stroke,M/F,AgeCategory,GenHealth,SleepTime
0,24.37,0,0,0,M,0,4,7.0
1,41.20,0,0,0,F,1,3,7.0
2,19.80,0,0,0,M,10,4,6.0
3,24.33,1,0,0,M,8,2,8.0
4,19.74,0,0,0,F,0,4,8.0


y: (2000,)
0    0
1    0
2    0
3    0
4    0
Name: HeartDisease, dtype: int64


In [4]:
print(y.value_counts())

HeartDisease
0    1000
1    1000
Name: count, dtype: int64


### ダミー変数化

In [5]:
X_dumm = pd.get_dummies(X, drop_first=True, dtype="uint8")
print("X_dumm:", X_dumm.shape)
display(X_dumm.head())

X_dumm: (2000, 8)


,BMI,Smoking,AlcoholDrinking,Stroke,AgeCategory,GenHealth,SleepTime,M/F_M
0,24.37,0,0,0,0,4,7.0,1
1,41.20,0,0,0,1,3,7.0,0
2,19.80,0,0,0,10,4,6.0,1
3,24.33,1,0,0,8,2,8.0,1
4,19.74,0,0,0,0,4,8.0,0


### トレーニングとテストのためにxとyを分割する

X_train, X_test, y_train, y_test = train_test_split(X_dumm, y, test_size=0.3, random_state=7, stratify=y)  
- X_train, X_test: 訓練用、テスト用の特徴データ
- y_train, y_test: 訓練用、テスト用の正解ラベル
  
- 第1引数：全入力データX
- 第2引数：全正解ラベルy
- test_size: テスト用の割合
- random_state: 乱数列を固定
- stratify=y: 元のyの各ラベルの比率を維持するようにtrain/testの分割が行われる

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dumm, y, test_size=0.3, random_state=7, stratify=y)


In [7]:
print(X_train.shape)
print(y_train.shape)
print(y_train.value_counts())


print(X_test.shape)
print(y_test.shape)
print(y_test.value_counts())

(1400, 8)
(1400,)
HeartDisease
0    700
1    700
Name: count, dtype: int64
(600, 8)
(600,)
HeartDisease
0    300
1    300
Name: count, dtype: int64


### ファイルから訓練用RFCモデルを読み込む
load(model_file)で学習済みモデルにする

In [8]:
tag = "heart_disease"

model_file = "rfc_{}.joblib".format(tag)
rfc = load(model_file)
print(rfc)

RandomForestClassifier(n_estimators=500, random_state=11)


### テストデータからの予測

In [10]:
y_test_pred = rfc.predict(X_test)
display(pd.crosstab(y_test, y_test_pred))


col_0,0,1
HeartDisease,,
0,229,71
1,92,208


In [11]:
acc = accuracy_score(y_test, y_test_pred)
print("Accuracy for test data: ",acc)

Accuracy for test data:  0.7283333333333334


In [12]:
X_test2 = pd.DataFrame([ [20.0, 0, 1, 0, 0, 4, 8.0, 0],
                         [70.0, 1, 0, 1, 12, 0, 4.0, 1],
                       ],
                       columns=X_dumm.columns)  # example
print('X for prediction:')
display(X_test2)

X for prediction:


,BMI,Smoking,AlcoholDrinking,Stroke,AgeCategory,GenHealth,SleepTime,M/F_M
0,20.0,0,1,0,0,4,8.0,0
1,70.0,1,0,1,12,0,4.0,1


In [13]:
y_test_pred_proba2 = rfc.predict_proba(X_test2)
y_test_pred2 = rfc.predict(X_test2)
print(y_test_pred_proba2)
print(y_test_pred2)

[[0.996 0.004]
 [0.078 0.922]]
[0 1]
